## 1.Environment Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from surprise import KNNWithMeans, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV, LeaveOneOut
from sklearn.metrics import confusion_matrix, classification_report, r2_score, precision_score, recall_score, f1_score
from collections import defaultdict, Counter

# Define path
data_path = '../datasets/student_grade.csv'



In [2]:
df = pd.read_csv(data_path, low_memory=False)
df

,cohort,acad_year,term,year_level,room,student_id,GEN223 DISASTER PREPAREDNESS,INT102 WEB TECHNOLOGY,INT20101 JAVASCRIPT PROGRAMMING,INT203 CLIENT-SIDE WEB PROGRAMMING II,...,INT491 SPECIAL TOPICS I : MULTIMEDIA TECHNOLOGY,INT492 SPECIAL TOPICS II : BUSINESS FINANCE AND DATA ANALYTICS,INT492 SPECIAL TOPICS II : DEVSECOPS,GEN354 DIGITAL PRODUCT INNOVATION AND COMMERCIALIZATION,INT530 SELECTED TOPICS IN INFRASTRUCTURE : INTERNET SECURITY (HANDS-ON APPROACH),INT540 SELECTED TOPICS IN INFORMATION TECHNOLOGY : DECISION SUPPORT SYSTEMS,LNG323 ENGLISH FOR DIGITAL SERVICE INNOVATION,SSC263 DEVELOPING LEADERS,INT420 BIG DATA ANALYTICS,INT491 SPECIAL TOPICS I : APPLIED COMPUTER VISION
0,66,2567,2,3,A,A375,0.0,0,0.0,2.5,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0
1,66,2567,2,3,A,A335,0.0,0,0.0,3.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0
2,66,2567,2,3,A,A381,0.0,0,0.0,2.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0
3,66,2567,2,3,A,A286,0.0,0,0.0,2.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0
4,66,2567,2,3,A,A515,0.0,0,2.5,2.5,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4298,63,2563,2,4,A,A442,0.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0
4299,63,2563,2,4,A,A479,0.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0
4300,63,2563,2,4,A,A545,0.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0
4301,63,2563,2,4,A,A658,0.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0


## 2.Data Loading & Preprocessing

In [ ]:
# === 2.1 Load Data ===
df = pd.read_csv(data_path, low_memory=False)

# === 2.2 Transform Data (Wide to Long) ===
id_vars = ['student_id']
df_long = pd.melt(df, id_vars=id_vars, var_name='course', value_name='grade')

# === 2.3 Clean Data ===
# Convert grade to numeric and remove invalid/empty grades
df_long['grade'] = pd.to_numeric(df_long['grade'], errors='coerce')
df_long_cleaned = df_long[(df_long['grade'] > 0.0) & (df_long['grade'].notna())].copy()

# === 2.4 Filter for 'INT' Courses Only ===
# This ensures the model only learns from INT courses
df_long_filtered = df_long_cleaned[df_long_cleaned['course'].astype(str).str.startswith('INT')].copy()

print(f"--- Data Preparation Complete ---")
print(f"Total records after cleaning: {len(df_long_cleaned)}")
print(f"Filtered to INT courses only: {len(df_long_filtered)}")
display(df_long_filtered)


File found. Loading data...
--- Data Preparation Complete ---
Total records after cleaning: 43104
Filtered to INT courses only: 17468


,student_id,course,grade
25871,A246,INT102 WEB TECHNOLOGY,1.5
27143,A632,INT102 WEB TECHNOLOGY,3.0
27144,A086,INT102 WEB TECHNOLOGY,4.0
27145,A662,INT102 WEB TECHNOLOGY,4.0
27146,A066,INT102 WEB TECHNOLOGY,2.5
...,...,...,...
592988,A385,INT491 SPECIAL TOPICS I : APPLIED COMPUTER VISION,3.5
592990,A424,INT491 SPECIAL TOPICS I : APPLIED COMPUTER VISION,2.5
593003,A111,INT491 SPECIAL TOPICS I : APPLIED COMPUTER VISION,2.5
593005,A627,INT491 SPECIAL TOPICS I : APPLIED COMPUTER VISION,2.5


## 3.Load Data and adjust rating scale

In [4]:
# Define rating scale (assuming grades are 1.0 to 4.0)
reader = Reader(rating_scale=(1, 4))
data = Dataset.load_from_df(df_long_filtered[['student_id', 'course', 'grade']], reader)

## 4.Parameter Tuning

In [ ]:
# Parameter grid สำหรับ KNNWithMeans
param_grid = {
    'k': [20, 40, 60],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['pearson', 'cosine'],   # pearson แนะนำสำหรับ WithMeans
        'user_based': [True, False]      # User–User / Item–Item
    }
}

gs = GridSearchCV(
    KNNWithMeans,
    param_grid,
    measures=['rmse', 'mae'],
    cv=5,
    joblib_verbose=3
)

print("🚀 Start GridSearchCV (KNNWithMeans)...")
gs.fit(data)
print("✅ GridSearchCV Finished")

print("\n🎯 Best RMSE:", gs.best_score['rmse'])
print("🏆 Best parameters:", gs.best_params['rmse'])


### RMSE DataFram

In [ ]:
# แปลงผลลัพธ์ทั้งหมดจาก Grid Search เป็น DataFrame
results_df = pd.DataFrame(gs.cv_results)

# แตก params dict ให้เป็นคอลัมน์
params_df = results_df['params'].apply(pd.Series)

# รวมเข้ากับ metric ที่สนใจ
df = pd.concat([
    params_df,
    results_df[['mean_test_rmse', 'mean_test_mae']]
], axis=1)

# แยกค่า name และ user_based ออกจาก sim_options
df['sim_name'] = df['sim_options'].apply(lambda x: x['name'])
df['user_based'] = df['sim_options'].apply(lambda x: x['user_based'])

print("Best RMSE score:", gs.best_score['rmse'])
print("Best params for RMSE:")
print(gs.best_params['rmse'])

print("\nBest MAE score:", gs.best_score['mae'])
print("Best params for MAE:")
print(gs.best_params['mae'])

plt.figure(figsize=(10, 6))

sns.lineplot(
    data=df,
    x='k',
    y='mean_test_rmse',
    hue='sim_name',
    marker='o',
    palette='viridis'
)

plt.title('Effect of k on RMSE for KNN')
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('RMSE (Lower is Better)')
plt.grid(True, alpha=0.3)
plt.show()


### RMSE Heatmap

In [ ]:
# -------------------------------------------------------
# Graph: Heatmap (k vs similarity metric)
# -------------------------------------------------------

# แปลงผล GridSearch ให้อยู่ในรูป DataFrame
results_df = pd.DataFrame.from_dict(gs.cv_results)

# NOTE:
# param_sim_options เป็น dict ดังนั้นต้องแตกค่า metric ออกมาก่อน
results_df['metric'] = results_df['param_sim_options'].apply(lambda x: x['name'])

# ทำ pivot table
pivot_table = results_df.pivot_table(
    values='mean_test_rmse',
    index='param_k',      # แกนตั้ง: k
    columns='metric'      # แกนนอน: similarity metric
)

plt.figure(figsize=(9, 6))
sns.heatmap(
    pivot_table,
    annot=True,
    fmt='.4f',
    cmap='Blues_r'   # ค่าสีเข้มกว่า = ดีกว่า (RMSE ต่ำกว่า)
)

plt.title('RMSE Heatmap (KNN) — k vs Similarity Metric')
plt.xlabel('Similarity Metric')
plt.ylabel('k')
plt.show()
